In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, log_loss
from sklearn.preprocessing import StandardScaler, LabelEncoder
from hyperopt import tpe,hp,Trials, space_eval
from hyperopt.fmin import fmin
import matplotlib.pyplot as plt

In [3]:
df_train = pd.read_csv('train.csv') #REDO
df_val = pd.read_csv('validation.csv')
df_test = pd.read_csv('test.csv')

X_train = df_train.drop('Cover_Type', axis=1)
y_train = df_train['Cover_Type']

X_val = df_val.drop('Cover_Type', axis=1)
y_val = df_val['Cover_Type']

X_test = df_test.drop('Cover_Type', axis=1)
y_test = df_test['Cover_Type']

In [4]:
def objective(params):
    lr=int(params['learning_rate'])
    md=int(params['max_depth'])
    g=int(params['gamma'])
    csbt=int(params['colsample_bytree'])
    ra=int(params['reg_alpha'])
    rl=int(params['reg_lambda'])

    model=xgb.XGBClassifier(learning_rate=lr, max_depth=md, gamma=g, colsample_bytree=csbt, reg_alpha=ra, reg_lambda=rl)
    model.fit(X_train,y_train)
    pred=model.predict(X_val)
    acc = accuracy_score(y_val, pred) 
    return acc

def optimize(trial):
    params={'learning_rate':hp.choice('learning_rate',[0.0001, 0.001, 0.01, 0.1, 1]),
           'max_depth':hp.choice('max_depth',range(3, 21)),
           'gamma':hp.choice('gamma',range(0,5)),
           'colsample_bytree':hp.choice('colsample_bytree',[i/10 for i in range(3,10)]),
           'reg_alpha':hp.choice('reg_alpha',[1e-5, 1e-2, 0.1, 1, 10, 100]),
           'reg_lambda':hp.choice('reg_lambda',[1e-5, 1e-2, 0.1, 1, 10, 100])
           }
    best_indices=fmin(fn=objective,space=params,algo=tpe.suggest,trials=trial,max_evals=100) # Possibly change max_evals and see if the results are much different
    
    # Convert the best indices to actual values
    best_hyperparameters = {
        'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1][best_indices['learning_rate']],
        'max_depth': range(3, 21)[best_indices['max_depth']],
        'gamma': range(0, 5)[best_indices['gamma']],
        'colsample_bytree': [i / 10 for i in range(3, 10)][best_indices['colsample_bytree']],
        'reg_alpha': [1e-5, 1e-2, 0.1, 1, 10, 100][best_indices['reg_alpha']],
        'reg_lambda': [1e-5, 1e-2, 0.1, 1, 10, 100][best_indices['reg_lambda']]
    }
    
    return best_hyperparameters

trial=Trials()
best=optimize(trial)

100%|██████████| 100/100 [00:32<00:00,  3.05trial/s, best loss: 0.7597113465376938]


In [15]:
best

{'learning_rate': 0.01,
 'max_depth': 16,
 'gamma': 1,
 'colsample_bytree': 0.4,
 'reg_alpha': 0.01,
 'reg_lambda': 10}

In [5]:
model = xgb.XGBClassifier(
    learning_rate=best['learning_rate'],
    max_depth=best['max_depth'],
    gamma=best['gamma'],
    colsample_bytree=best['colsample_bytree'],
    reg_alpha=best['reg_alpha'],
    reg_lambda=best['reg_lambda']
)

model.fit(X_train, y_train)

yhat = model.predict(X_test)
yhat_probabilities = model.predict_proba(X_test)

accuracy = accuracy_score(y_test, yhat)
recall = recall_score(y_test, yhat, average='weighted')
f1 = f1_score(y_test, yhat, average='weighted')
loss = log_loss(y_test, yhat_probabilities)

print("Loss: ", loss)
print("Accuracy: ", accuracy)
print("Recall: ", recall)
print("F1: ", f1)

Loss:  0.6881646137454486
Accuracy:  0.85283459246975
Recall:  0.85283459246975
F1:  0.8368724081814236


In [6]:
model2 = xgb.XGBClassifier()

model2.fit(X_train, y_train)
yhat = model2.predict(X_test)
yhat_probabilities = model2.predict_proba(X_test)

accuracy = accuracy_score(y_test, yhat)
recall = recall_score(y_test, yhat, average='weighted')
f1 = f1_score(y_test, yhat, average='weighted')
loss = log_loss(y_test, yhat_probabilities)

print("Loss: ", loss)
print("Accuracy: ", accuracy)
print("Recall: ", recall)
print("F1: ", f1)

Loss:  0.284313711358974
Accuracy:  0.8716909280756711
Recall:  0.8716909280756711
F1:  0.8677917717023028


In [7]:
model2.get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': None,
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}